## Task 3: Prompt Engineering for Large Language Models (LLMs) [4 marks]

### Zero-shot and Few-Shot Prompting

**Zero-shot prompting** involves providing a language model with a prompt or a set of instructions that allows it to generate text or perform a task without any explicit training data or labeled examples. The model is expected to generate high-quality text or perform the task accurately based solely on the prompt and its internal knowledge.

**Few-shot prompting** is similar to zero-shot prompting, but it involves providing the model with a limited number of labeled examples or prompts that are relevant to the specific task or dataset. The model is then expected to generate high-quality text or perform the task accurately based on the few labeled examples and its internal knowledge.

### Task Description

You have been provided with a Python notebook that demonstrates how to use zero-shot and few-shot prompting with a language model (LLM). The example in the notebook involves text-based tasks, but LLMs can also be applied to a wide range of tasks (Students interested in learning more can read [here](https://link-to-resource) and [here](https://link-to-resource)).

Queries will be provided in the form of featurized accelerometer data and the model should predict the activity performed.

- **Zero-shot learning:** The model should be able to predict the activity based on the accelerometer data without any explicit training data or labeled examples.
- **Few-shot learning:** The model should also be able to predict the activity based on a limited number of labeled examples or prompts that are relevant to the specific task.

## Questions

1. **Demonstrate how to use Zero-Shot Learning and Few-Shot Learning to classify human activities based on the featurized accelerometer data.** Qualitatively demonstrate the performance of Few-Shot Learning with Zero-Shot Learning. Which method performs better? Why? [1 mark]

2. **Quantitatively compare the accuracy of Few-Shot Learning with Decision Trees** (You may use a subset of the test set if you encounter rate-limiting issues). Which method performs better? Why? [1 mark]

3. **What are the limitations of Zero-Shot Learning and Few-Shot Learning** in the context of classifying human activities based on featurized accelerometer data? [1 mark]

4. **What does the model classify when given input from an entirely new activity that it hasn't seen before?** [0.5 mark]

5. **Test the model with random data** (ensuring the data has the same dimensions and range as the previous input) and report the results. [0.5 mark]

In [2]:
import pandas as pd
import os

# Define the path to the UCI HAR dataset
uci_base_path = r'C:\Users\arpit\OneDrive\Desktop\es335-24-fall-assignment-1\UCI HAR Dataset\Combined'

# Define the activities
activities = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']

# Initialize lists to store the data and labels
X_train = []
y_train = []

# Process each activity
for activity in activities:
    activity_path = os.path.join(uci_base_path, 'train', activity)
    for file_name in os.listdir(activity_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(activity_path, file_name)
            data = pd.read_csv(file_path)

            # Select the relevant columns
            data = data[['accx', 'accy', 'accz']]

            # Append DataFrame to X_train without converting to NumPy
            X_train.append(data)
            y_train.extend([activity] * data.shape[0])  # Append the label for each sample

# Concatenate all DataFrames in X_train to form a single DataFrame
X_train = pd.concat(X_train, axis=0)
y_train = pd.Series(y_train)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Display the first few rows to confirm column names
print(X_train.head())
print(y_train.head())


X_train shape: (470528, 3)
y_train shape: (470528,)
       accx      accy      accz
0  1.426164 -0.362485  0.278914
1  1.496596 -0.591127  0.120137
2  1.305815 -0.645547  0.012587
3  0.973824 -0.543838 -0.001186
4  0.691378 -0.424250 -0.015278
0    WALKING
1    WALKING
2    WALKING
3    WALKING
4    WALKING
dtype: object


In [3]:
import numpy as np
import pandas as pd
from langchain_groq import ChatGroq
import os
from sklearn.tree import DecisionTreeClassifier
from dotenv import load_dotenv

load_dotenv()

Groq_Token = os.getenv('GROQ_API_KEY')
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}
model_name = "llama3.1-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
# Testing data 

file1_laying =  pd.read_csv("Combined/Test/LAYING/Subject_2.csv")
file2_walking = pd.read_csv("Combined/Test/WALKING/Subject_2.csv")
file3_sitting=  pd.read_csv("Combined/Test/SITTING/Subject_2.csv")
file4_standing=  pd.read_csv("Combined/Test/STANDING/Subject_2.csv")
file5_upstairs=  pd.read_csv("Combined/Test/WALKING_UPSTAIRS/Subject_2.csv")
file6_downstairs=  pd.read_csv("Combined/Test/WALKING_DOWNSTAIRS/Subject_2.csv")


df1 = pd.DataFrame(file1_laying).head(100)
df2 = pd.DataFrame(file2_walking).head(100)
df3 = pd.DataFrame(file3_sitting).head(100)
df4 = pd.DataFrame(file4_standing).head(100)
df5 = pd.DataFrame(file5_upstairs).head(100)
df6 = pd.DataFrame(file6_downstairs).head(100)


# Training Data for few shot prompt examples

laying_test = pd.read_csv("Combined/Train/LAYING/Subject_1.csv")
sitting_test = pd.read_csv("Combined/Train/SITTING/Subject_1.csv")
standing_test = pd.read_csv("Combined/Train/STANDING/Subject_1.csv")
walking_test = pd.read_csv("Combined/Train/WALKING/Subject_1.csv")
downstairs_test = pd.read_csv("Combined/Train/WALKING_DOWNSTAIRS/Subject_1.csv")
upstairs_test = pd.read_csv("Combined/Train/WALKING_UPSTAIRS/Subject_1.csv")

laying_df = pd.DataFrame(laying_test).head(100)
sitting_df = pd.DataFrame(sitting_test).head(100)
standing_df = pd.DataFrame(standing_test).head(100)
walking_df = pd.DataFrame(walking_test).head(100)
downstairs_df = pd.DataFrame(downstairs_test).head(100)
upstairs_df = pd.DataFrame(upstairs_test).head(100)




In [4]:
# Zero shot demonstration

zero_shot_prompt = f"""
* You are a human activity recognition model.
* Your task is to classify the following accelerometer data into one of the six activities: Walking, Standing, Sittting, Laying, Walking Upstairs, Walking Downstairs. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.
Here is the accelerometer data:
{df1}, {df2}, {df3}

Please classify the activity for these three accelerometer data.
"""

zero_shot_answer = llm.invoke(zero_shot_prompt)
print(zero_shot_answer.content)

Based on the provided accelerometer data, I will classify the activities as follows:

**Data 1:**
The accelerometer data shows a relatively stable pattern with small variations in the x, y, and z axes. The values are mostly within a small range, indicating a low level of movement. This pattern is consistent with the activity of **Standing**.

**Data 2:**
The accelerometer data shows a significant variation in the x-axis, with values ranging from approximately 0.7 to 1.15. This suggests a high level of movement in the x-axis, which is consistent with the activity of **Walking**. The y and z axes show relatively smaller variations, which further supports this classification.

**Data 3:**
The accelerometer data shows a relatively stable pattern with small variations in the x, y, and z axes. However, the values are slightly higher than those in Data 1, indicating a slightly higher level of movement. This pattern is consistent with the activity of **Sitting**. The data does not show the hig

In [5]:
# Few Shot demonstration
few_shot_prompt = f""" 
* You are a human activity recognition model.
* Your task is to classify the following accelerometer data into one of the six activities: Walking, Standing, Sittting, Laying, Walking Upstairs, Walking Downstairs. 
* Provide only labels for the dataset. 

Here are some examples:
1.Dataset of laying: {laying_df}
2.Dataset of sitting: {sitting_df}
3.Dataset of standing: {standing_df}
4.Dataset of walking: {walking_df}
5.Dataset of walking downstairs: {downstairs_df}
6.Dataset of walking upstairs: {upstairs_df}

Here is the accelerometer data:
{df1}, 
{df2},
{df3},
{df4},
{df5},
{df6}

Please classify the activity for these six accelerometer data using the dataset of sample activites.
"""
few_shot_answer = llm.invoke(few_shot_prompt)
print(few_shot_answer.content)


Based on the provided accelerometer data, I will classify the activities as follows:

1. Laying
2. Walking
3. Standing
4. Walking
5. Walking Downstairs
6. Walking Upstairs



Q-1: 

In our current model, ZSL is performing more, however in ideal case FSL should perfom well, as it takes few examples first and then works on data. Zero-Shot Learning (ZSL) can perform well if the language model has strong general knowledge and the prompt is clear and detailed. If the accelerometer data is straightforward and aligns well with the model’s existing understanding, ZSL might provide accurate results even without task-specific examples. Therefore, ZSL’s performance can sometimes match or exceed Few-Shot Learning in specific cases. 

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

true_labels = ["Laying", "Sitting","Standing", "Walking" , "Walking Downstairs", "Walking Upstairs"]
model_predictions = ["Laying", "Walking", "Standing", "Walking", "Walking Downstairs", "Walking Upstairs"]

accuracy = accuracy_score(true_labels, model_predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 83.33%


In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

true_labels = ["Laying", "Walking","Sitting"]
model_predictions = ["Standing", "Walking","Sitting"]

accuracy = accuracy_score(true_labels, model_predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 66.67%




 

Q-2: 

Decision Trees might perform better because they can clearly learn and use patterns in the data to make accurate predictions. They work well with structured data and can achieve high accuracy if the features are well-represented. Few-Shot Learning, while flexible, may need more specific examples to reach the same level of accuracy. If Decision Trees perfectly match the data's patterns, they will likely outperform Few-Shot Learning in terms of accuracy. 

 

Q-3: 

Limitations of Zero-Shot Learning (ZSL): 

-Limited Accuracy: ZSL may not always provide accurate results if the model's general knowledge doesn't perfectly match the task. 
-Prompt Dependence: Its performance heavily relies on the quality and clarity of the prompt. 
-Lack of Specificity: Without task-specific examples, the model might struggle with nuances in the data. 

Limitations of Few-Shot Learning (FSL): 

-Requires Examples: FSL needs a few labeled examples, which may not always be available. 
-Example Quality: The model’s performance depends on the relevance and quality of the provided examples. 
-Limited Generalization: FSL might struggle with generalizing beyond the few examples if they don’t cover the full range of possible data patterns. 

 

Q-4: 

When given input from an entirely new activity that the model hasn't seen before, it might classify the activity as "Unknown" or select the closest matching known activity based on its training. In Zero-Shot Learning, the model relies on general knowledge and prompts, potentially leading to less accurate or vague classifications. In Few-Shot Learning, the model might struggle to classify the new activity accurately if it hasn't been well-represented in the few examples provided.

In [7]:
import pandas as pd
import numpy as np

def create_random_data(num_rows=70):
    np.random.seed(42)
    x_accel = np.random.uniform(0.1697406, 0.2500059, num_rows)
    y_accel = np.random.uniform(0.4911341, 0.5668622, num_rows)
    z_accel = np.random.uniform(0.7768708, 0.8653713, num_rows)
    return pd.DataFrame({'x_accel': x_accel, 'y_accel': y_accel, 'z_accel': z_accel})

activity_data = {
    "Laying": create_random_data(),
    "Walking": create_random_data(),
    "Sitting": create_random_data(),
    "Standing": create_random_data(),
    "Walking_Upstairs": create_random_data(),
    "Walking_Downstairs": create_random_data()
}
data_csv_strings = {key: df.head(100).to_csv(index=False) for key, df in activity_data.items()}

zero_shot_prompt = f"""
* You are a model for human activity recognition.
* Classify the following accelerometer data into one of these activities: Walking, Standing, Laying.
* Provide the classification and a brief explanation if needed.
* Ensure that each output is different.
Here is the accelerometer data:
1. {data_csv_strings['Walking']}
2. {data_csv_strings['Standing']}
3. {data_csv_strings['Laying']}

Please classify the activity for these three sets of accelerometer data.
"""

zero_shot_response = llm.invoke(zero_shot_prompt)
print(zero_shot_response.content)

few_shot_prompt = f""" 
* You are a model for human activity recognition.
* Classify the following accelerometer data into one of these activities: Walking, Sitting, Laying.
* Provide only the labels.
* Ensure that each output is different.

Here are sample datasets:
1. Data for Laying: {activity_data['Laying']}
2. Data for Sitting: {activity_data['Sitting']}
3. Data for Walking: {activity_data['Walking']}

Here is the new accelerometer data:
{data_csv_strings['Walking']}, 
{data_csv_strings['Sitting']},
{data_csv_strings['Laying']}
Please classify the activity for these three new datasets using the provided examples.
"""

few_shot_response = llm.invoke(few_shot_prompt)
print(few_shot_response.content)


Based on the provided accelerometer data, I will classify each set of data into one of the three activities: Walking, Standing, or Laying.

**Set 1:**
The data shows a relatively high variation in the x and y axes, with a range of approximately 0.1-0.3 g. The z-axis values are also relatively high, with a range of approximately 0.8-0.9 g. This pattern is consistent with walking, as the x and y axes capture the movement of the legs and arms, while the z-axis captures the vertical movement of the body.

**Classification:** Walking

**Set 2:**
The data shows a similar pattern to Set 1, with a high variation in the x and y axes and relatively high z-axis values. However, the range of the x and y axes is slightly smaller, and the z-axis values are slightly more consistent. This could indicate a slightly slower pace or a more stable walking pattern.

**Classification:** Walking

**Set 3:**
The data shows a very similar pattern to Set 1 and Set 2, with a high variation in the x and y axes and